In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib

# Load datasets
stations = pd.read_csv("stations.csv")
station_day = pd.read_csv("station_day.csv")

# Merge datasets on 'station'
df = pd.merge(station_day, stations, on="StationId", how="left")

print("Data Loaded Successfully")
print("Shape:", df.shape)
df.head()

Data Loaded Successfully
Shape: (108035, 20)


,StationId,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket,StationName,City,State,Status
0,AP001,2017-11-24,71.36,115.75,1.75,20.65,12.40,12.19,0.10,10.76,109.26,0.17,5.92,0.10,NaN,NaN,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
1,AP001,2017-11-25,81.40,124.50,1.44,20.50,12.08,10.72,0.12,15.24,127.09,0.20,6.50,0.06,184.0,Moderate,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
2,AP001,2017-11-26,78.32,129.06,1.26,26.00,14.85,10.28,0.14,26.96,117.44,0.22,7.95,0.08,197.0,Moderate,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
3,AP001,2017-11-27,88.76,135.32,6.60,30.85,21.77,12.91,0.11,33.59,111.81,0.29,7.63,0.12,198.0,Moderate,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active
4,AP001,2017-11-28,64.18,104.09,2.56,28.07,17.01,11.42,0.09,19.00,138.18,0.17,5.02,0.07,188.0,Moderate,"Secretariat, Amaravati - APPCB",Amaravati,Andhra Pradesh,Active


In [11]:
# Drop duplicate rows
df = df.drop_duplicates()

# Drop rows with missing target (AQI)
df = df.dropna(subset=["AQI"])

# Fill numeric columns with median
num_cols = ["PM2.5", "PM10", "NO2", "SO2", "CO", "O3"]
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Fill categorical columns with mode
cat_cols = ["City", "State"]
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# Ensure types
df[cat_cols] = df[cat_cols].astype(str)

# Final check
print("Cleaned Data Summary:")
print(df.isnull().sum())
print("Final Shape:", df.shape)

Cleaned Data Summary:
StationId          0
Date               0
PM2.5              0
PM10               0
NO              2229
NO2                0
NOx             4555
NH3            29832
CO                 0
SO2                0
O3                 0
Benzene        19787
Toluene        26324
Xylene         67584
AQI                0
AQI_Bucket         0
StationName        0
City               0
State              0
Status           311
dtype: int64
Final Shape: (87025, 20)


In [12]:
# Select features and target
features = ["PM2.5", "PM10", "NO2", "SO2", "CO", "O3", "City", "State"]
target = "AQI"

X = df[features]
y = df[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Split complete — Train:", X_train.shape, "Test:", X_test.shape)

Split complete — Train: (69620, 8) Test: (17405, 8)


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import ExtraTreesRegressor

# Define feature types
numeric_features = ["PM2.5", "PM10", "NO2", "SO2", "CO", "O3"]
categorical_features = ["City", "State"]

# Numeric preprocessing
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Categorical preprocessing
categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Combine both preprocessors
preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

# Create full Extra Trees pipeline
et_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", ExtraTreesRegressor(n_estimators=200, random_state=42))
])

# Train
print("🔹 Training Extra Trees...")
et_pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred_et = et_pipeline.predict(X_test)
r2_et = r2_score(y_test, y_pred_et)
mae_et = mean_absolute_error(y_test, y_pred_et)
rmse_et = np.sqrt(mean_squared_error(y_test, y_pred_et))

print(f"\nExtra Trees Results:")
print(f"R²: {r2_et:.4f} | MAE: {mae_et:.2f} | RMSE: {rmse_et:.2f}")

# Save model
joblib.dump(et_pipeline, "extra_trees_aqi.pkl")
print("Model saved as extra_trees_aqi.pkl")

🔹 Training Extra Trees...

Extra Trees Results:
R²: 0.9190 | MAE: 21.28 | RMSE: 37.40
Model saved as extra_trees_aqi.pkl


In [13]:
import tensorflow as tf
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# --- Prepare data ---
feature_cols = ["PM2.5", "PM10", "NO2", "SO2", "CO", "O3"]
X_seq = df[feature_cols].values
y_seq = df["AQI_next"].values

# Split 80/20
split = int(0.8 * len(X_seq))
X_train_seq, X_test_seq = X_seq[:split], X_seq[split:]
y_train_seq, y_test_seq = y_seq[:split], y_seq[split:]

# --- Define Transformer-like model using Dense layers ---
def build_st_transformer(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    return model

# Initialize model
input_dim = len(feature_cols)
model = build_st_transformer(input_dim)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mse',
              metrics=['mae'])

# --- Training ---
print("\nTraining ST-Transformer (TensorFlow)...")
history = model.fit(
    X_train_seq, y_train_seq,
    validation_split=0.1,
    epochs=25,
    batch_size=32,
    verbose=1
)

print("ST-Transformer training complete.")

# --- Evaluation ---
y_pred_st = model.predict(X_test_seq).flatten()

r2_st = r2_score(y_test_seq, y_pred_st)
mae_st = mean_absolute_error(y_test_seq, y_pred_st)
rmse_st = np.sqrt(mean_squared_error(y_test_seq, y_pred_st))

print(f"\nST-Transformer Results:")
print(f"R²: {r2_st:.4f} | MAE: {mae_st:.2f} | RMSE: {rmse_st:.2f}")

# --- Save model ---
model.save("st_transformer_aqi_tf.keras")
print("Model saved as st_transformer_aqi_tf.keras")



Training ST-Transformer (TensorFlow)...
Epoch 1/25
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 1s 400us/step - loss: 7939.4839 - mae: 55.9256 - val_loss: 1257.8563 - val_mae: 25.3841
Epoch 2/25
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 1s 385us/step - loss: 3994.5872 - mae: 41.7722 - val_loss: 1124.8641 - val_mae: 22.2539
Epoch 3/25
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 1s 404us/step - loss: 3430.4199 - mae: 36.8815 - val_loss: 1080.1593 - val_mae: 20.7293
Epoch 4/25
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 1s 407us/step - loss: 3170.3997 - mae: 34.7538 - val_loss: 1019.3242 - val_mae: 19.4798
Epoch 5/25
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 1s 370us/step - loss: 3084.7217 - mae: 34.2389 - val_loss: 1125.8759 - val_mae: 21.6938
Epoch 6/25
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 1s 366us/step - loss: 3053.7761 - mae: 33.9276 - val_loss: 1023.5617 - val_mae: 19.3902
Epoch 7/25
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 1s 367us/step - loss: 3017.7490 - mae: 33.6697 - val_loss: 1081.3049 - val_mae: 20.4988
Epoch 8/25
1956/1956 ━━━━━━━━━━━━━━━━━━━━ 1s 367u

In [14]:
print("\n Comparing Models...")

print(f"ST-Transformer → R²: {r2_st:.4f}, MAE: {mae_st:.2f}, RMSE: {rmse_st:.2f}")
print(f"Extra Trees    → R²: {r2_et:.4f}, MAE: {mae_et:.2f}, RMSE: {rmse_et:.2f}")

if r2_st > r2_et:
    best_model_name = "ST-Transformer"
    joblib.dump({"type": "STTransformer", "path": "st_transformer_aqi.pth"}, "best_model.pkl")
else:
    best_model_name = "Extra Trees"
    joblib.dump(et_pipeline, "best_model.pkl")

print(f"\n Best Model: {best_model_name} (saved as best_model.pkl)")


 Comparing Models...
ST-Transformer → R²: 0.8448, MAE: 26.56, RMSE: 40.75
Extra Trees    → R²: 0.9190, MAE: 21.28, RMSE: 37.40

 Best Model: Extra Trees (saved as best_model.pkl)


In [13]:
import joblib
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Features
numeric_features = ["PM2.5", "PM10", "NO2", "SO2", "CO", "O3"]
categorical_features = ["City", "State"]

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

et_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", ExtraTreesRegressor(n_estimators=200, random_state=42))
])

print("🔹 Training Extra Trees model again...")
et_pipeline.fit(X_train, y_train)

# Evaluate
y_pred = et_pipeline.predict(X_test)
print(f"R²: {r2_score(y_test, y_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}")

# Export proper joblib bundle
all_features = numeric_features + categorical_features
joblib.dump(
    {"model": et_pipeline, "columns": all_features},
    "best_model.joblib",
    compress=3
)

print("✅ Exported best_model.joblib successfully!")


🔹 Training Extra Trees model again...
R²: 0.9190
MAE: 21.28
RMSE: 37.40
✅ Exported best_model.joblib successfully!


In [1]:
import pandas as pd

# Load your dataset (change filename if needed)
df = pd.read_csv("stations.csv")  # or pd.read_excel("your_file.xlsx")

# Get unique cities
unique_cities = df["City"].dropna().unique()

# Get unique states
unique_states = df["State"].dropna().unique()

print("Unique Cities:")
print(unique_cities)

print("\nUnique States:")
print(unique_states)


Unique Cities:
['Amaravati' 'Rajamahendravaram' 'Tirupati' 'Vijayawada' 'Visakhapatnam'
 'Guwahati' 'Gaya' 'Hajipur' 'Muzaffarpur' 'Patna' 'Chandigarh' 'Delhi'
 'Ahmedabad' 'Ankleshwar' 'Gandhinagar' 'Nandesari' 'Vapi' 'Vatva'
 'Ambala' 'Bahadurgarh' 'Ballabgarh' 'Bhiwani' 'Dharuhera' 'Faridabad'
 'Fatehabad' 'Gurugram' 'Hisar' 'Jind' 'Kaithal' 'Karnal' 'Kurukshetra'
 'Mandikhera' 'Manesar' 'Narnaul' 'Palwal' 'Panchkula' 'Panipat' 'Rohtak'
 'Sirsa' 'Sonipat' 'Yamuna Nagar' 'Jorapokhar' 'Bagalkot' 'Bengaluru'
 'Chamarajanagar' 'Chikkaballapur' 'Chikkamagaluru' 'Hubballi'
 'Kalaburagi' 'Mysuru' 'Ramanagara' 'Vijayapura' 'Yadgir' 'Eloor'
 'Ernakulam' 'Kannur' 'Kochi' 'Kollam' 'Kozhikode' 'Thiruvananthapuram'
 'Bhopal' 'Damoh' 'Dewas' 'Gwalior' 'Indore' 'Jabalpur' 'Katni' 'Maihar'
 'Mandideep' 'Pithampur' 'Ratlam' 'Sagar' 'Satna' 'Singrauli' 'Ujjain'
 'Aurangabad' 'Chandrapur' 'Kalyan' 'Mumbai' 'Nagpur' 'Nashik'
 'Navi Mumbai' 'Pune' 'Solapur' 'Thane' 'Shillong' 'Aizawl' 'Brajrajnagar'
 'T